In [43]:
import pandas as pd
import pickle
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
from scipy.fft import fft, fftfreq
from Bio import Entrez, SeqIO, AlignIO, pairwise2, Align, Seq, motifs
from Bio.Seq import Seq
from Bio.SeqFeature import SeqFeature, FeatureLocation
from scipy.stats import binom
import math
import random
from tqdm.auto import tqdm
import operator

In [44]:
cryptic_output_path = "D:/Project_Data/CRYPTIC_DATA/Cryptic_Data_Analysis"
cryptic_input_path = "D:/Project_Data/CRYPTIC_DATA/"

In [45]:
genome_info = pd.read_csv(cryptic_input_path + 'genomes.csv')[['UNIQUEID','MYKROBE_LINEAGE_NAME_1', 'MYKROBE_LINEAGE_NAME_2']]
genome_info.fillna('Missing', inplace=True)
genome_counts = (genome_info.groupby(['MYKROBE_LINEAGE_NAME_2'])[['UNIQUEID']].count()).reset_index()
genome_counts_dict = dict(zip(genome_counts.MYKROBE_LINEAGE_NAME_2, genome_counts.UNIQUEID))
genome_info_dict = {}
for i, r in genome_info.iterrows():
    genome_info_dict[r['UNIQUEID']] = r['MYKROBE_LINEAGE_NAME_2']

C:\Users\nicho\AppData\Local\Temp/ipykernel_25124/4031226401.py:1: DtypeWarning: Columns (1,3,13,14,15,27,36) have mixed types. Specify dtype option on import or set low_memory=False.
  genome_info = pd.read_csv(cryptic_input_path + 'genomes.csv')[['UNIQUEID','MYKROBE_LINEAGE_NAME_1', 'MYKROBE_LINEAGE_NAME_2']]
C:\Users\nicho\AppData\Local\Temp/ipykernel_25124/4031226401.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  genome_info.fillna('Missing', inplace=True)


In [46]:
chunksize = 0.5 * (10 ** 7) 
with pd.read_csv(cryptic_input_path + "VARIANTS.csv", chunksize=chunksize) as reader:
    chunknum = 0
    for chunk in tqdm(reader):
        variants_sample_df = chunk[['UNIQUEID', 'MUTATION_TYPE', 'GENOME_INDEX', 'REF', 'ALT']]
        variants_sample_df = variants_sample_df[variants_sample_df['MUTATION_TYPE'] == 'SNP']
        variants_sample_df['MYKROBE_LINEAGE_NAME_2'] = variants_sample_df['UNIQUEID'].map(genome_info_dict);
        temp = variants_sample_df.groupby(['GENOME_INDEX', 'MYKROBE_LINEAGE_NAME_2','REF', 'ALT'])[['UNIQUEID']].count()
        temp = temp.reset_index()
        if chunknum == 0:
            variant_summary_df = temp
        else:
            variant_summary_df = pd.concat([variant_summary_df, temp])
        chunknum += 1

temp = variant_summary_df.groupby(['GENOME_INDEX', 'MYKROBE_LINEAGE_NAME_2','REF','ALT'])[['UNIQUEID']].sum()
variant_summary_df = temp.reset_index()        
variant_summary_df['LINEAGE_COUNT'] = variant_summary_df['MYKROBE_LINEAGE_NAME_2'].map(genome_counts_dict)
variant_summary_df.rename(columns = {'UNIQUEID':'MUTATION_COUNT'}, inplace = True)
variant_summary_df.to_csv(cryptic_output_path + '/variant_summary_df.csv')

0it [00:00, ?it/s]

C:\Users\nicho\Anaconda3\envs\Bioinformatics\lib\site-packages\tqdm\std.py:1195: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
C:\Users\nicho\Anaconda3\envs\Bioinformatics\lib\site-packages\tqdm\std.py:1195: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:
C:\Users\nicho\Anaconda3\envs\Bioinformatics\lib\site-packages\tqdm\std.py:1195: DtypeWarning: Columns (28) have mixed types. Specify dtype option on import or set low_memory=False.
  for obj in iterable:


In [47]:
filter_list = ['Unknown','Missing']
filtered_variant_summary_df = variant_summary_df.query('not(MYKROBE_LINEAGE_NAME_2 == @filter_list) &  LINEAGE_COUNT > 200')
filtered_variant_summary_df['MUTATION_PCT'] = filtered_variant_summary_df['MUTATION_COUNT'] / filtered_variant_summary_df['LINEAGE_COUNT']
filtered_variant_summary_df.to_csv(cryptic_output_path + '/filtered_variant_summary_df.csv')

C:\Users\nicho\AppData\Local\Temp/ipykernel_25124/1689673011.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_variant_summary_df['MUTATION_PCT'] = filtered_variant_summary_df['MUTATION_COUNT'] / filtered_variant_summary_df['LINEAGE_COUNT']


In [48]:
filtered_variant_summary_df[filtered_variant_summary_df['GENOME_INDEX'] == 1093322]

,GENOME_INDEX,MYKROBE_LINEAGE_NAME_2,REF,ALT,MUTATION_COUNT,LINEAGE_COUNT,MUTATION_PCT
1253044,1093322,lineage4,c,t,12,342,0.035088
1253055,1093322,lineage4.3.1,c,t,281,281,1.000000
1253056,1093322,lineage4.3.2,c,t,801,807,0.992565
1253057,1093322,lineage4.3.2.1,c,t,1772,1783,0.993831
1253062,1093322,lineage4.3.3,c,t,3295,3319,0.992769
1253066,1093322,lineage4.3.4.1,c,t,635,637,0.996860
1253069,1093322,lineage4.3.4.2,c,t,1046,1053,0.993352
1253070,1093322,lineage4.3.4.2.1,c,t,1515,1534,0.987614
